In [123]:
import pandas as pd
import folium
import geopy
from geopy.distance import great_circle
import deap
import random
#import mlrose
import numpy as np
import plotly.express as px


In [2]:
df_final_cities = pd.read_csv('/Users/admin/Desktop/GitHub/new_repos/jingle-jam-2023/notebooks/final_cities.csv', encoding = "iso-8859-1")


In [3]:
display(df_final_cities)

,City,Latitude,Longitude
0,Athens,37.9838,23.7275
1,Cairo,30.0444,31.2357
2,Hialeah,25.8576,-80.2781
3,Lincoln,40.8136,-96.7026
4,Cleveland,41.4993,-81.6944
5,Bangkok,13.7563,100.5018
6,Gilbert,33.3528,-111.7890
7,Corpus Christi,27.8006,-97.3964
8,Osaka,34.6937,135.5022


In [4]:
def calculate_distance(location1, location2):
    return great_circle(location1, location2).mi

name_1 = df_final_cities.iloc[0, 0]
lat_long_1 = df_final_cities.iloc[0, 1], df_final_cities.iloc[0, 2]

name_2 = df_final_cities.iloc[1, 0]
lat_long_2 = df_final_cities.iloc[1, 1], df_final_cities.iloc[1, 2]

distance = round(calculate_distance(lat_long_1, lat_long_2), 0)
print(f"Distance between {name_1} and {name_2}: {distance} miles")

Distance between Athens and Cairo: 696.0 miles


In [5]:
number_list = [0,1,2,3,4,5,6,7,8]

random.shuffle(number_list)
print(number_list)
df_final_cities['number'] = number_list


[1, 8, 5, 3, 6, 0, 4, 2, 7]


In [6]:
display(df_final_cities)

,City,Latitude,Longitude,number
0,Athens,37.9838,23.7275,1
1,Cairo,30.0444,31.2357,8
2,Hialeah,25.8576,-80.2781,5
3,Lincoln,40.8136,-96.7026,3
4,Cleveland,41.4993,-81.6944,6
5,Bangkok,13.7563,100.5018,0
6,Gilbert,33.3528,-111.7890,4
7,Corpus Christi,27.8006,-97.3964,2
8,Osaka,34.6937,135.5022,7


In [19]:
def fitness(route):
    total_distance = 0
    for i in range(len(route) - 1):

        location1 = route[i]
        location2 = route[i+1]

        lat_long_1 = (df_final_cities.loc[df_final_cities['number'] == location1, 'Latitude'].iloc[0], 
                      df_final_cities.loc[df_final_cities['number'] == location1, 'Longitude'].iloc[0])
        lat_long_2 = (df_final_cities.loc[df_final_cities['number'] == location2, 'Latitude'].iloc[0], 
                      df_final_cities.loc[df_final_cities['number'] == location2, 'Longitude'].iloc[0])

        total_distance += calculate_distance(lat_long_1, lat_long_2)
    return total_distance

In [30]:
def create_population():
    location_numbers = list(range(9))

    population_size = 10 
    population = []

    for pop_number in range(population_size):
        optimized_route = location_numbers[:]
        random.shuffle(optimized_route)
        population.append(optimized_route)
    return population


In [31]:
initial_population = create_population()
for i in initial_population:
    result = fitness(i)
    print(result)

37696.96239421576
35597.96664786186
32347.485613039757
33532.238592803806
47708.77145827713
40200.600598253586
41933.54354635344
47459.976987761256
42980.35854976476
28830.722594438626


In [35]:
def selection_optimize(population, fitness_func, tournament_size=10):
    selected = []
    for j in range(len(population)):
        contenders = random.sample(population, tournament_size)
        winner = min(contenders, key=fitness_func)
        selected.append(winner)
    return selected

In [43]:
def ordered_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    offspring = [None] * size
    offspring[start:end] = parent1[start:end]
    fill_values = [item for item in parent2 if item not in offspring]
    for i in range(size):
        if offspring[i] is None:
            offspring[i] = fill_values.pop(0)
    return offspring

In [49]:
def swap_mutation(route, mutation_rate):
    mutated_route = route[:]
    for i in range(len(route)):
        if random.random() < mutation_rate:
            swap_index = random.randint(0, len(route) - 1)
            mutated_route[i], mutated_route[swap_index] = mutated_route[swap_index], mutated_route[i]
    return mutated_route

In [121]:
num_generations = 250 
population_size = 10  
mutation_rate = 0.1  
best_route_per_generation = []

# Initialize population with random routes
selection_population = create_population()

for generation in range(num_generations):

    fitness_scores = [fitness(route) for route in selection_population]

    optimized_selection = selection_optimize(selection_population, fitness, tournament_size=5)

    next_generation = []
    while len(next_generation) < population_size:
        route_1, route_2 = random.sample(optimized_selection, 2)
        crossover_route = ordered_crossover(route_1, route_2)
        next_generation.append(crossover_route)

    population = [swap_mutation(route, mutation_rate) for route in next_generation]

    best_route = min(population, key=fitness)
    best_route_fitness = fitness(best_route)
    best_route_per_generation.append((best_route, best_route_fitness))



In [122]:
best_overall_route = min(best_route_per_generation, key=lambda x: x[1])
print(best_overall_route)

([7, 0, 8, 1, 5, 2, 4, 3, 6], 17421.24144777315)
